In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
    
from models.trainer import *
import numpy as np
import torch
from models.spectro_transformer import *
from data.dataset import *
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset

/home/hutiany4/miniconda3/envs/a2a-music-gen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wdir = os.getcwd().replace("\\", "/") + "/.."
processed_dir = f"{wdir}/data/processed/musicnet"

spec_dataset = SpectrogramDataset(
    f"{processed_dir}/train_data", split_ratio=0.9
)

token_size = spec_dataset[0][0].shape[0]
seq_size = spec_dataset[0][0].shape[1]
out_size = spec_dataset[0][1].shape[1]

train_set, val_set = random_split(spec_dataset, [0.8, 0.2])

train_loader = DataLoader(train_set, batch_size=5, shuffle=True)
val_loader = DataLoader(val_set, batch_size=5, shuffle=True)

for x, y, _ in train_loader:
    batch_size, token_len, seq_len = x.shape[0], x.shape[1], x.shape[2]
    print(f"token_len: {token_len}, seq_len: {seq_len}")
    break

token_len: 201, seq_len: 744


In [3]:
configuration = ASTConfig()
configuration.num_mel_bins = token_len # dimension size of frequency
configuration.max_length = seq_len      # dimension size of input time length
configuration.hidden_dropout_prob = 0.2
configuration.hidden_size = 60
configuration.num_hidden_layers = 1
model = AST(configuration)

In [4]:
trainer = AST_trainer()
trainer.set_dataloaders(train_loader=train_loader, val_loader=val_loader, test_loader=None)
trainer.model = model
history = trainer.train(1e-4, 1e-2, 10)


Validation epoch 0: 100%|██████████| 13/13 [00:18<00:00,  1.40s/it]



Epoch: 0 	Training Loss: 31.5586 	Validation Loss: 29.0290
	 	Training Accuracy: 30.09% 	Validation Accuracy: 32.49%


Validation epoch 1: 100%|██████████| 13/13 [00:18<00:00,  1.39s/it]



Epoch: 1 	Training Loss: 31.5217 	Validation Loss: 28.9913
	 	Training Accuracy: 31.61% 	Validation Accuracy: 32.65%


Train epoch 2:  54%|█████▍    | 28/52 [01:53<01:36,  4.04s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

val_loss = history["val_loss"].to_list()
train_loss = history["train_loss"].to_list()
val_acc = history["val_acc"].to_list()
train_acc = history["train_acc"].to_list()
x = np.arange(history.shape[0])

fig = plt.figure(0)
plt.plot(x, train_loss)
fig = plt.figure(1)
plt.plot(x, val_loss)
fig = plt.figure(2)
plt.plot(x, train_acc)
fig = plt.figure(3)
plt.plot(x, val_acc)
plt.show()